In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

from tqdm.notebook import trange, tqdm

### Train the agent
* Run it until he has a running average above the success_threshold
* Use a large number of episodes for the running average ( 1000+ ) so if even it falls into a privileged sample, it wont be prone to error

In [2]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
        use_technical_indicators= [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
    ])
    
    env.success_threshold =0.25 # 25%
    return env

# from src.environments.continuous.trading import environment

In [3]:
from src.agents.ppo import PpoAgent

In [4]:
agent=PpoAgent(environment)
agent.load()
environment().success_threshold


[Errno 2] Unable to open file (unable to open file: name = 'storage/environments/AaplStockTradingEnvironment-v0/e73112700f4e9dcda3624d3d2d671e68/models/ppo-actor.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

[Errno 2] Unable to open file (unable to open file: name = 'storage/environments/AaplStockTradingEnvironment-v0/e73112700f4e9dcda3624d3d2d671e68/models/ppo-critic.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)




0.25

In [5]:
obs = agent.env.reset()
obs.shape

(21, 33)

In [6]:
np.expand_dims(obs,axis=0).shape

(1, 21, 33)

In [7]:
agent.actor.predict(np.expand_dims(obs,axis=0))

array([[1., 0., 0.]], dtype=float32)

In [8]:
agent.act(np.expand_dims(obs,axis=0))

(0, array([1., 0., 0.]), array([1., 0., 0.], dtype=float32))

In [ ]:
agent.learn(
    timesteps=-1, 
    log_every=100,
    success_threshold_lookback=1000,
    success_strict=True,
)

episode: 1/100000, score: 0.0, average: 0.00 
episode: 2/100000, score: 0.0, average: 0.00 
episode: 3/100000, score: 0.0, average: 0.00 
episode: 4/100000, score: 0.0, average: 0.00 
episode: 5/100000, score: 0.0, average: 0.00 
episode: 6/100000, score: 0.0, average: 0.00 
episode: 7/100000, score: 0.0, average: 0.00 
episode: 8/100000, score: 0.0, average: 0.00 
episode: 9/100000, score: 0.0, average: 0.00 
episode: 10/100000, score: 0.0, average: 0.00 
episode: 11/100000, score: 0.0, average: 0.00 
episode: 12/100000, score: 0.0, average: 0.00 
episode: 13/100000, score: 0.0, average: 0.00 
episode: 14/100000, score: 0.0, average: 0.00 
episode: 15/100000, score: 0.0, average: 0.00 
episode: 16/100000, score: 0.0, average: 0.00 
episode: 17/100000, score: 0.0, average: 0.00 
episode: 18/100000, score: 0.0, average: 0.00 
episode: 19/100000, score: 0.0, average: 0.00 
episode: 20/100000, score: 0.0, average: 0.00 
episode: 21/100000, score: 0.0, average: 0.00 
episode: 22/100000, sc

episode: 139/100000, score: 0.08558463809328713, average: 0.08 
episode: 140/100000, score: -0.14538186489766386, average: 0.07 
episode: 141/100000, score: -0.1146533148333612, average: 0.06 
episode: 142/100000, score: -0.07795261545380344, average: 0.06 
episode: 143/100000, score: -0.1567945696906585, average: 0.05 
episode: 144/100000, score: 0.1113659489148467, average: 0.06 
episode: 145/100000, score: 0.23044604056173826, average: 0.06 
episode: 146/100000, score: 0.23044604056173826, average: 0.05 
episode: 147/100000, score: -0.12765428385631358, average: 0.05 
episode: 148/100000, score: 0.17028320283676654, average: 0.05 
episode: 149/100000, score: 0.19530502719197737, average: 0.05 
episode: 150/100000, score: -0.07167269049797209, average: 0.05 
episode: 151/100000, score: -0.11616950239284936, average: 0.04 
episode: 152/100000, score: -0.11616950239284936, average: 0.04 
episode: 153/100000, score: 0.17129899289100056, average: 0.05 
episode: 154/100000, score: 0.10775

### Test the results
* Runs a set of episodes with unseen data
* Stores the results in a csv file for later consulting

In [ ]:
success = 0
n_tests = 10000

scores = []
targets = []
portfolio_target_ratios = []
initial_investments = []

for i in trange(n_tests):
    state = agent.env.reset(visualize=False,mode="test")
    step = 0
    score = 0
    reward = 0
    done = False
    
    targets.append((agent.env.episode_target-agent.env.initial_investment)/agent.env.initial_investment)
    initial_investments.append(agent.env.initial_investment)

    while not done:
        agent.env.render()
        #state = np.expand_dims(state, axis=0)
        action, action_onehot, prediction = agent.act(state)
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, info = agent.env.step(action)
        #print(action, reward, agent.env.portfolio_value)
        # Memorize (state, action, reward) for training
        #self.buffer.remember(np.expand_dims(state, axis=0), action_onehot, reward)
        # Update current state
        if done :
            if agent.env.portfolio_value > agent.env.initial_investment:
                success +=1

        step+=1
        state = next_state
        score += reward
    
    # Track scores and ratios
    scores.append(score)
    portfolio_target_ratios.append(info["portfolio_value"]/info["episode_target"] -1)
    
    
test_results_dataframe = pd.DataFrame([[
    n_tests,
    str(round(np.mean(scores)*100,3))+'%',
    str(round(np.mean(targets)*100,3))+'%',
    str(round(np.mean(portfolio_target_ratios)*100,3))+'%',
    str(round(min(scores)*100,3))+'%',
    str(round(max(scores)*100,3))+'%',
    str(round((success/n_tests)*100,3)) +'%'
]],
    columns=[
        '# Blind tests',
        '% Average portfolio return', 
        '% Desired portfolio return', 
        'Portfolio/Target rate',
        '% Historical minimum return',
        '% Historical maximum return', 
        '% Episodes concluded with positive outcome'
    ]
)



test_results_dataframe.T

In [ ]:
agent.results_writer.store_test_results(agent,test_results_dataframe)

### Visual test
* Runs a set of episodes with unseen data
* See the evolution in real time


In [ ]:
success = 0
n_tests = 2

scores = []
targets = []


for i in trange(n_tests):
    state = agent.env.reset(visualize=True,mode="test")
    step = 0
    score = 0
    reward = 0
    done = False
    targets.append((agent.env.episode_target-agent.env.initial_investment)/agent.env.initial_investment)
    initial_portfolio = agent.env.portfolio_value

    while not done:
        agent.env.render()
        #state = np.expand_dims(state, axis=0)
        action, action_onehot, prediction = agent.act(state)
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)
        #print(action, reward, agent.env.portfolio_value)
        # Memorize (state, action, reward) for training
        #self.buffer.remember(np.expand_dims(state, axis=0), action_onehot, reward)
        # Update current state
        if done :
            if agent.env.portfolio_value > agent.env.initial_investment:
                success +=1

        step+=1
        state = next_state
        score += reward
    
    #print(score,initial_portfolio, agent.env.portfolio_value)
    scores.append(score)

agent.env.close()

In [ ]:
e = environment()

In [ ]:
obs = e.observation_space.sample()

In [ ]:
np.reshape(obs, [1, obs.shape])

In [ ]:
np.reshape(1,obs.shape)

In [ ]:
(1,2).append(1)

In [ ]:
#np.reshape([1,2,3])

In [ ]:
obs.shape